In [1]:
import pandas as pd
import matplotlib.pyplot as plt

text_comments Data Loading + Cleaning

In [2]:
text_reader = pd.read_csv("data/text_comments.csv", iterator=True, chunksize=1000000,lineterminator='\n')

In [ ]:
# text_comments.csv is very large, so going to split it into 2 time-based dataframes: before and after Feb 1, 2020 
# Each dataframe will have id, linkid, author, and body
# The dataframes will not contain usernames but we will also remove rows with 
# Also, if a comment has body [deleted] or [removed] then, we remove those as well
# It's easier to deal with these smaller datasets but if necessary, we can concatenate these pre and post covid dataframes later on

In [3]:
df_comments_pre = pd.DataFrame() # pre-covid (before Feb 1, 2020)
df_comments_post = pd.DataFrame() # post-covid (Feb 1, 2020 and later)
temp = pd.DataFrame()

df_comments = pd.DataFrame()
lst = []

In [4]:
for chunk in text_reader: 
    # Add each chunk of txt_comments to a df and append it to a list
    temp = pd.DataFrame()
    temp = chunk[["id", "link_id", "author", "body", "created_utc"]]
    lst.append(temp)

    



    # remove usernames that indicate it's a bot and comments that are [removed] and [deleted]
    # temp = temp[~((temp["body"] == "[removed]") | (temp["body"] == "[deleted]") | temp["author"].str.lower().str.contains("bot|mod"))]
    # temp=temp.drop(columns=['author'])
    
    # temp = temp['created_utc'].astype(int)

    # df_comments = pd.concat([df_comments,temp])

    # Pre-Covid Dataframe
    # df_comments_pre = pd.concat([df_comments_pre, temp[temp["created_utc"] < 1580515200]])

    # Post-Covid Dataframe
    # df_comments_post = pd.concat([df_comments_pre, temp[temp["created_utc"] >= 1580515200]])

In [29]:
len(lst)

41

In [31]:
for i in range(len(lst)): # TODO 0 to len
    print(i)
    lst[i] = lst[i][~((lst[i]["body"] == "[removed]") | (lst[i]["body"] == "[deleted]") | lst[i]["author"].str.lower().str.contains("bot|mod"))]
    lst[i] = lst[i].drop(columns=['author'])
    lst[i]['created_utc'] = lst[i]['created_utc'].astype(int)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [55]:
df_comments_pre = pd.concat([df_comments_pre, lst[1][lst[1]["created_utc"] < 1580515200]])

In [56]:
df_comments_post = pd.concat([df_comments_post, lst[1][lst[1]["created_utc"] >= 1580515200]])

In [66]:
for i in range(2, len(lst)): # TODO 0 to len
    print(i)
        
    # Pre-Covid Dataframe
    df_comments_pre = pd.concat([df_comments_pre, lst[i][lst[i]["created_utc"] < 1580515200]])

    # Post-Covid Dataframe
    df_comments_post = pd.concat([df_comments_post, lst[i][lst[i]["created_utc"] >= 1580515200]])

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [ ]:
# See if the shape[0] of every element in lst = df_pre.shape[0] + df_post.shape[0]

In [68]:
df_comments_pre.shape # (13635058, 4)
df_comments_post.shape # (22814508, 4)

(22814508, 4)

In [46]:
df_comments_pre

""


In [69]:
df_comments_pre

,id,link_id,body,created_utc
3058,t1_fg500j1,t3_ewseet,I really love how precise she is with makeup a...,1580514977
3062,t1_fd2eb6n,t3_ejq7ek,That part of the whole story is extremely susp...,1578145985
3069,t1_fd3xequ,t3_ejuxan,"Prep: If it's a constant-work period, I let pe...",1578162447
3070,t1_fd4llkb,t3_ek1pnm,Can you repost linking the actual video and no...,1578168690
3076,t1_fd63yhw,t3_ek3ifu,This gave me weird al vibes and I’m living for...,1578185200
...,...,...,...,...
40721545,t1_ew3s0p8,t3_cmccmo,"Did people forget Finn had to face Corbin, Jin...",1565092947
40721546,t1_ennpzrt,t3_bp2f3w,I hope,1557949685
40721547,t1_f02psre,t3_d3f1tn,I will literally cry if you're right dude xD L...,1568335635
40721548,t1_f5t4bhw,t3_dp6jz9,Wow. That's incredibly hostile for no reason.,1572450141


In [70]:
df_comments_post

,id,link_id,body,created_utc
0,t1_ftjl56l,t3_gzv6so,Does anyone have a good cruelty free one? The ...,1591755558
1,t1_ftjpxmc,t3_gzv6so,(stares at my soft glam i've had for like 3 ye...,1591758382
2,t1_gzzxfyt,t3_nodb9e,When Jen’s initial reactions came out to the s...,1622398357
3,t1_gzzy7nc,t3_no6qaj,I think you mean a \n>Highschool *fucking* bully,1622398743
4,t1_h00tpbp,t3_nolx7p,Ugh. I was honestly hoping that this brand wou...,1622414834
...,...,...,...,...
26926619,t1_gsytzd2,t3_mhgs9x,Getting Rocky mountain high......,1617230525
26926620,t1_gsz18gx,t3_mhi4wh,Same with any drug,1617234409
26926621,t1_gljb5rw,t3_l9n3m1,"I bought it, ngl it feels weird",1612136949
26926622,t1_g78g4w0,t3_j2pjuo,Welp guess I found an excuse to smoke today,1601510378


In [ ]:
# check the sizes are the same: 40721549 index. 40721550 rows total

In [71]:
df_comments_pre.to_csv('data/df_comments_pre.csv')

In [ ]:
df_comments_post.to_csv('data/df_comments_post.csv')

text_submissions Data Loading + Cleaning

In [10]:
text_submissions_df = pd.read_csv("data/text_submissions.csv")

C:\Users\farih\AppData\Local\Temp\ipykernel_11164\2287351077.py:1: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  text_submissions_df = pd.read_csv("data/text_submissions.csv")


In [11]:
text_submissions_df

,id,author,created_utc,domain,is_self,score,selftext,title,subreddit
0,t3_npxigk,All_Consuming_Void,1622563615,self.BeautyGuruChatter,True,0.0,[removed],Hyram launches his own brand,BeautyGuruChatter
1,t3_nqj6bf,AutoModerator,1622631621,self.BeautyGuruChatter,True,38.0,What are the influencers trying to influence y...,What I'm not gonna buy Wednesday - Anti-haul,BeautyGuruChatter
2,t3_nk0btr,barrahhhh,1621869439,reddit.com,False,144.0,NaN,Plouise goes off in facebook group for 'bullying',BeautyGuruChatter
3,t3_nrbybs,[deleted],1622722260,self.BeautyGuruChatter,True,2.0,[deleted],Is youtube algorithm against Susan Yara? She g...,BeautyGuruChatter
4,t3_nl0ebd,carlosShook,1621977767,vm.tiktok.com,False,0.0,NaN,Sephora steals concept from Huntr Faulknr afte...,BeautyGuruChatter
...,...,...,...,...,...,...,...,...,...
3496175,t3_ae5pzt,middlefinger22,1547030976,self.yakuzagames,True,1.0,And they won't have so much changes as Kiwami2.,"Why it takes so long to release 3, 4 and 5 on ...",yakuzagames
3496176,t3_cer9jj,AutoModerator,1563449362,self.nrl,True,15.0,|NRL| ...,Round 18: Broncos vs Bulldogs | Post Match Dis...,nrl
3496177,t3_bqidb1,nuke8960,1558280683,i.redd.it,False,35.0,NaN,Gotta spread the word,BirdsArentReal
3496178,t3_cqvbwk,anon3212,1565900167,self.ShadowBan,True,1.0,Am I shadowbanned?,Test,ShadowBan


In [12]:
# Dropping columns that aren't useful for us to make the dataframes smaller

text_submissions_df=text_submissions_df.drop(columns=['score'])

In [13]:
# Drop this row because it has an invalid date and other columns are nan
text_submissions_df=text_submissions_df[~(text_submissions_df['created_utc'] == "CPTSD")]

In [14]:
text_submissions_df['created_utc'] = text_submissions_df['created_utc'].astype(int)

In [15]:
# start of the pandemic as of February 1, 2020
# created_at is a UNIX timestamp. Feb 1, 2020 00:00:00 UTC = 1580515200
text_submissions_df_pre = text_submissions_df[text_submissions_df["created_utc"] < 1580515200]
text_submissions_df_post = text_submissions_df[text_submissions_df["created_utc"] >= 1580515200]

In [84]:
# TODO (Q): Bot Detection not necessary for Posts? just for text_comments?

In [ ]:
# Downloading the 4 updated cvs which have cleaned up the data and split the comments and submissions csv files into 2 
# because they are smaller files, thus easier to load up and deal with
# text_submissions_df_pre.to_csv('data/text_submissions_pre.csv')
# text_submissions_df_post.to_csv("data/text_submissions_post.csv")